In [5]:
import time
import pyautogui
import subprocess
import os
from PIL import Image

def bring_app_to_front(app_name):
    """
    指定したアプリケーションを前面に表示させる関数。
    
    :param app_name: 例 "Google Chrome" や "Safari" など
    """
    script = f'tell application "{app_name}" to activate'
    subprocess.call(["osascript", "-e", script])


def capture_screenshots(max_pages=10, pause_time=2, path="", direction="right",app_name= "Arc",region=None):
    """
    アクティブなウィンドウで右矢印キーを送信しながらスクリーンショットを取得する関数。
    
    :param max_pages: 取得するページ数（最大）
    :param pause_time: ページ遷移後の待機時間（秒）
    :param region: スクリーンショットを取得する領域（left, top, width, height）を指定する場合
    """
    bring_app_to_front(app_name)
    time.sleep(1)  # ウィンドウが前面に来るまで少し待つ
    
    pyautogui.click(x=500, y=300)  # クリックしてウィンドウをアクティブにする
    
    for i in range(max_pages):
        screenshot_path = f'{path}/screenshot_{i:03d}.png'
        screenshot = pyautogui.screenshot(region=region)
        screenshot.save(screenshot_path)
        print(f"Captured: {screenshot_path}")
        pyautogui.press(direction)  # 右矢印キーの入力をシミュレート
        time.sleep(pause_time)

# スクショを終えたファイルをまとめてPDF化するメソッド
def convert_images_to_pdf(image_dir, pdf_path):
    """
    指定されたディレクトリ内のPNG画像を読み込み、1つのPDFにまとめる関数。

    :param image_dir: PNG画像が保存されているディレクトリのパス
    :param pdf_path: 出力するPDFファイルのパス（例: "AIエディタCursor完全ガイド.pdf"）
    """
    # 画像ファイル一覧を取得（拡張子を小文字に統一してフィルタ）
    image_files = sorted(
        [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.lower().endswith('.png')]
    )
    if not image_files:
        print("PNG画像が見つかりませんでした:", image_dir)
        return

    image_list = []
    for image_file in image_files:
        img = Image.open(image_file).convert('RGB')
        image_list.append(img)
    
    if image_list:
        # 最初の画像に残りの画像を追加してPDF化
        image_list[0].save(pdf_path, save_all=True, append_images=image_list[1:])
        print(f"PDFが保存されました: {pdf_path}")


if __name__ == "__main__":
    # 対象ウィンドウをあらかじめ手動で前面に表示させておくこと
    site = 'screenshots/Kindle/'
    book = 'AIエディタCursor完全ガイド/'
    path = f'{site}{book}'
    max_pages = 300
    direction = 'right'
    app_name = "Amazon Kindle" # Arc
    if not os.path.exists(path):
        os.makedirs(path)
    
    capture_screenshots(max_pages=max_pages, pause_time=1, path=path, direction=direction, app_name=app_name)
    
    book_name = os.path.basename(os.path.normpath(path))
    pdf_output = os.path.join(path, f"{book_name}.pdf")
    convert_images_to_pdf(image_dir=path, pdf_path=pdf_output)

KeyboardInterrupt: 